In [ ]:
pip install dotenv

In [39]:
import gradio as gr
from dotenv import load_dotenv
import urllib
import google.generativeai as genai

load_dotenv() # load "/.env" file.

genai.configure() # load GOOGLE_API_KEY from "/.env" file


# 메인 페이지입니다.
def main_page():
    with gr.Column() as main_view:
        gr.Markdown("## 메인 페이지")
        gr.Markdown("사진으로 음식을 알아보고 조리 과정을 알아보거나, 집에 있는 재료로 어떤 요리를 만들지 알아보세요!")
        cook_page_btn = gr.Button("사진으로 음식 알아보기")
        ingredient_btn = gr.Button("집에 있는 재료로 만들 수 있는 음식은?")
    return main_view, cook_page_btn, ingredient_btn

# process
def process(image, prompt) :
    try:
        vmodel = genai.GenerativeModel('gemini-pro-vision')
        response = vmodel.generate_content([prompt, image])
        return response.text
    except Exception as e:
        return f"Error {e}"

# 조리 제조 과정 페이지. (기능 1)
def cook_page():
    with gr.Column(visible=False) as cook_page:
        gr.Markdown("## 사진으로 음식을 알아보고 조리에 필요한 준비와 조리 과정을 알아보세요!")
        gr.Interface(process, [gr.Image(type="pil"), "text"], "text")
        cook_to_main_btn = gr.Button("메인 페이지로")
    return cook_page, cook_to_main_btn

# 재료로 음식 추천 받기. (기능 2)
def ingredient_page():
    with gr.Column(visible=False) as ingredient_page:
        gr.Markdown("## 집에 있는 재료로 어떤 음식을 만들 수 있는지 알아보세요!")
        ingredient_to_main_btn = gr.Button("메인 페이지로")
    return ingredient_page, ingredient_to_main_btn

# MAIN FUNCITON #
def main():
    with gr.Blocks() as app:
        main_view, cook_page_btn, ingredient_btn = main_page()
        cook_view, cook_to_main_btn = cook_page()
        ingredient_view, ingredient_to_main_btn= ingredient_page()

        def gr_update(main_visible, cook_visible, ingredient_visible):
            return gr.update(visible=main_visible), gr.update(visible=cook_visible), gr.update(visible=ingredient_visible)

        def show_main_page():
            return gr_update(True, False, False)
        
        def show_cook_page():
            return gr_update(False, True, False)
    
        def show_ingredient_page():
            return gr_update(False, False, True)
        output = [main_view, cook_view, ingredient_view]

        # button events
        cook_page_btn.click(show_cook_page, [], output)
        ingredient_btn.click(show_ingredient_page, [], output)
        cook_to_main_btn.click(show_main_page, [], output)
        ingredient_to_main_btn.click(show_main_page, [], output)
        

    app.launch()

if __name__ == "__main__":
    main()

Running on local URL:  http://127.0.0.1:7897

To create a public link, set `share=True` in `launch()`.
